In [1]:
%%writefile Student.java
public class Student {
    private int rollNumber;
    private String name;
    private String department;

    public Student(int rollNumber, String name, String department) {
        this.rollNumber = rollNumber;
        this.name = name;
        this.department = department;
    }

    public int getRollNumber() { return rollNumber; }
    public String getName() { return name; }
    public String getDepartment() { return department; }

    @Override
    public String toString() {
        return rollNumber + "," + name + "," + department;
    }

    public static Student fromString(String line) {
        String[] p = line.split(",");
        return new Student(
            Integer.parseInt(p[0]),
            p[1],
            p[2]
        );
    }
}


Writing Student.java


In [2]:
%%writefile Seat.java
public class Seat {
    private int row;
    private int col;
    private Student student;

    public Seat(int row, int col) {
        this.row = row;
        this.col = col;
    }

    public void assignStudent(Student student) {
        this.student = student;
    }

    public Student getStudent() {
        return student;
    }

    @Override
    public String toString() {
        if (student == null)
            return "[ EMPTY ]";
        return "[ " + student.getRollNumber() + " - " + student.getDepartment() + " ]";
    }
}


Writing Seat.java


In [3]:
%%writefile ExamHall.java
public class ExamHall {
    private String hallName;
    private int rows;
    private int cols;
    private Seat[][] seats;

    public ExamHall(String hallName, int rows, int cols) {
        this.hallName = hallName;
        this.rows = rows;
        this.cols = cols;

        seats = new Seat[rows][cols];
        for (int i = 0; i < rows; i++)
            for (int j = 0; j < cols; j++)
                seats[i][j] = new Seat(i, j);
    }

    public String getHallName() { return hallName; }
    public int getCapacity() { return rows * cols; }
    public Seat[][] getSeats() { return seats; }

    public void displayHall() {
        System.out.println("\n=== Seating for Hall: " + hallName + " ===");
        for (Seat[] row : seats) {
            for (Seat s : row) {
                System.out.print(s + " ");
            }
            System.out.println();
        }
    }
}


Writing ExamHall.java


In [4]:
%%writefile SeatingAlgorithm.java
import java.util.*;

public class SeatingAlgorithm {

    public static void arrangeSeating(List<Student> students, List<ExamHall> halls) {

        // Shuffle students for randomness
        Collections.shuffle(students);

        int studentIndex = 0;

        for (ExamHall hall : halls) {
            Seat[][] seats = hall.getSeats();

            for (int i = 0; i < seats.length; i++) {
                for (int j = 0; j < seats[i].length; j++) {

                    if (studentIndex >= students.size())
                        return;

                    Student current = students.get(studentIndex);

                    // Check anti-cheating rule: no adjacent same department
                    if (isSafe(seats, i, j, current)) {
                        seats[i][j].assignStudent(current);
                        studentIndex++;
                    } else {
                        // Try next student
                        int nextIndex = findNextSafeStudent(students, studentIndex, seats, i, j);
                        if (nextIndex != -1) {
                            Student temp = students.get(nextIndex);
                            students.set(nextIndex, current);
                            students.set(studentIndex, temp);
                            seats[i][j].assignStudent(temp);
                            studentIndex++;
                        }
                    }
                }
            }
        }
    }

    private static boolean isSafe(Seat[][] seats, int r, int c, Student s) {
        if (r > 0 && seats[r-1][c].getStudent() != null &&
            seats[r-1][c].getStudent().getDepartment().equals(s.getDepartment()))
            return false;

        if (c > 0 && seats[r][c-1].getStudent() != null &&
            seats[r][c-1].getStudent().getDepartment().equals(s.getDepartment()))
            return false;

        return true;
    }

    private static int findNextSafeStudent(List<Student> list, int start, Seat[][] seats, int r, int c) {
        for (int i = start + 1; i < list.size(); i++) {
            if (isSafe(seats, r, c, list.get(i)))
                return i;
        }
        return -1;
    }
}


Writing SeatingAlgorithm.java


In [5]:
%%writefile FileManager.java
import java.io.*;
import java.util.*;

public class FileManager {

    public static List<Student> loadStudents(String filename) {
        List<Student> students = new ArrayList<>();

        File file = new File(filename);
        if (!file.exists()) return students;

        try (BufferedReader br = new BufferedReader(new FileReader(file))) {
            String line;
            while ((line = br.readLine()) != null) {
                students.add(Student.fromString(line));
            }
        } catch (Exception e) {
            System.out.println("Error loading students.");
        }

        return students;
    }

    public static void saveSeatingChart(String filename, List<ExamHall> halls) {
        try (PrintWriter pw = new PrintWriter(new FileWriter(filename))) {
            for (ExamHall hall : halls) {
                pw.println("\n=== Hall: " + hall.getHallName() + " ===");
                Seat[][] seats = hall.getSeats();

                for (Seat[] row : seats) {
                    for (Seat s : row)
                        pw.print(s + " ");
                    pw.println();
                }
            }

            System.out.println("Seating chart saved to " + filename);

        } catch (Exception e) {
            System.out.println("Error saving seating chart.");
        }
    }
}


Writing FileManager.java


In [6]:
%%writefile Main.java
import java.util.*;

public class Main {
    public static void main(String[] args) {

        Scanner sc = new Scanner(System.in);

        // Load students from file
        List<Student> students = FileManager.loadStudents("students.txt");

        if (students.isEmpty()) {
            System.out.println("No students found in file 'students.txt'");
            return;
        }

        // Create halls
        List<ExamHall> halls = new ArrayList<>();
        halls.add(new ExamHall("Hall A", 3, 4));
        halls.add(new ExamHall("Hall B", 3, 4));

        // Arrange seating
        SeatingAlgorithm.arrangeSeating(students, halls);

        // Display seating
        for (ExamHall hall : halls)
            hall.displayHall();

        // Save chart
        FileManager.saveSeatingChart("seating_output.txt", halls);
    }
}


Writing Main.java


In [7]:
%%writefile students.txt
101,Ram,CSE
102,John,IT
103,Sita,ECE
104,Kiran,CSE
105,Asha,ECE
106,Raju,IT
107,Meera,CSE
108,Arun,ECE
109,Vijay,IT
110,Rohit,CSE


Writing students.txt


In [8]:
!javac *.java


In [9]:
!java Main



=== Seating for Hall: Hall A ===
[ 103 - ECE ] [ 106 - IT ] [ 107 - CSE ] [ 105 - ECE ] 
[ 110 - CSE ] [ 108 - ECE ] [ 109 - IT ] [ 104 - CSE ] 
[ 102 - IT ] [ 101 - CSE ] [ EMPTY ] [ EMPTY ] 

=== Seating for Hall: Hall B ===
[ EMPTY ] [ EMPTY ] [ EMPTY ] [ EMPTY ] 
[ EMPTY ] [ EMPTY ] [ EMPTY ] [ EMPTY ] 
[ EMPTY ] [ EMPTY ] [ EMPTY ] [ EMPTY ] 
Seating chart saved to seating_output.txt
